In [8]:
import librosa
import numpy as np
import pandas as pd
import os

def extraer_features_egg(path, n_mfcc=13):
    try:
        y, sr = librosa.load(path, sr=None)  # preserva 50kHz

        features = {}
        features['file_name'] = os.path.basename(path)
        features['duration'] = librosa.get_duration(y=y, sr=sr)

        # MFCCs
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        for i in range(n_mfcc):
            features[f'MFCC_{i+1}_mean'] = np.mean(mfccs[i])
            features[f'MFCC_{i+1}_std'] = np.std(mfccs[i])

        # Brillo espectral (Spectral Centroid)
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        features['spectral_centroid_mean'] = np.mean(centroid)
        features['spectral_centroid_std'] = np.std(centroid)

        # Inharmonicidad (Spectral Flatness)
        flatness = librosa.feature.spectral_flatness(y=y)
        features['spectral_flatness_mean'] = np.mean(flatness)

        # Aperiodicidad aproximada (Zero Crossing Rate)
        zcr = librosa.feature.zero_crossing_rate(y)
        features['zcr_mean'] = np.mean(zcr)

        # HNR aproximado con librosa (relación armónico/ruido)
        S, phase = librosa.magphase(librosa.stft(y))
        harm = librosa.effects.harmonic(y)
        noise = librosa.effects.percussive(y)
        hnr = 10 * np.log10(np.sum(harm ** 2) / (np.sum(noise ** 2) + 1e-6))
        features['hnr_librosa'] = hnr

        return features

    except Exception as e:
        print(f"❌ Error procesando {path}: {e}")
        return None

def procesar_eggs_en_directorio(raiz='healthy'):
    data = []
    contador = 0
    for root, dirs, files in os.walk(raiz):
        for archivo in files:
            if archivo.endswith('-egg.egg'):
                print(f'procesando {archivo}')
                path = os.path.join(root, archivo)
                feats = extraer_features_egg(path)
                if feats:
                    feats['path'] = path
                    data.append(feats)

    df = pd.DataFrame(data)
    return df

In [9]:
df_egg = procesar_eggs_en_directorio('./voice-audios/svd_dataset/unhealthy')
df_egg.to_csv("features_svd_unhealthy.csv", index=False)


procesando 1086-a_h-egg.egg


C:\Users\noefa\AppData\Local\Temp\ipykernel_56084\2427092168.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=None)  # preserva 50kHz


procesando 1086-a_l-egg.egg
procesando 1086-a_lhl-egg.egg
procesando 1086-a_n-egg.egg
procesando 1086-iau-egg.egg
procesando 1086-i_h-egg.egg
procesando 1086-i_l-egg.egg
procesando 1086-i_lhl-egg.egg
procesando 1086-i_n-egg.egg
procesando 1086-u_h-egg.egg
procesando 1086-u_l-egg.egg
procesando 1086-u_lhl-egg.egg
procesando 1086-u_n-egg.egg
procesando 1194-a_h-egg.egg
procesando 1194-a_l-egg.egg
procesando 1194-a_lhl-egg.egg
procesando 1194-a_n-egg.egg
procesando 1194-iau-egg.egg
procesando 1194-i_h-egg.egg
procesando 1194-i_l-egg.egg
procesando 1194-i_lhl-egg.egg
procesando 1194-i_n-egg.egg
procesando 1194-u_h-egg.egg
procesando 1194-u_l-egg.egg
procesando 1194-u_lhl-egg.egg
procesando 1194-u_n-egg.egg
procesando 1219-a_h-egg.egg
procesando 1219-a_l-egg.egg
procesando 1219-a_lhl-egg.egg
procesando 1219-a_n-egg.egg
procesando 1219-iau-egg.egg
procesando 1219-i_h-egg.egg
procesando 1219-i_l-egg.egg
procesando 1219-i_lhl-egg.egg
procesando 1219-i_n-egg.egg
procesando 1219-u_h-egg.egg
proc

In [2]:
import os
import librosa
import numpy as np
import pandas as pd

def procesar_eggs_en_directorio_mel(raiz='healthy', n_mels=128):
    data = []

    for root, dirs, files in os.walk(raiz):
        for archivo in files:
            if archivo.endswith('-egg.egg'):
                print(f'🔊 Procesando {archivo}')
                path = os.path.join(root, archivo)

                try:
                    y, sr = librosa.load(path, sr=None)
                    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
                    S_dB = librosa.power_to_db(S, ref=np.max)

                    data.append({
                        'file_name': archivo,
                        'duration': librosa.get_duration(y=y, sr=sr),
                        'sr': sr,
                        'mel_shape': S_dB.shape,
                        'mel_matrix': S_dB   
                    })

                except Exception as e:
                    print(f"❌ Error procesando {archivo}: {e}")

    df = pd.DataFrame(data)
    return df



In [4]:
df = procesar_eggs_en_directorio_mel('./voice-audios/svd_dataset/unhealthy')
df.to_pickle("mel_svd_unhealthy.pkl")

🔊 Procesando 1086-a_h-egg.egg
🔊 Procesando 1086-a_l-egg.egg
🔊 Procesando 1086-a_lhl-egg.egg


C:\Users\noefa\AppData\Local\Temp\ipykernel_63372\3595633242.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=None)


🔊 Procesando 1086-a_n-egg.egg
🔊 Procesando 1086-iau-egg.egg
🔊 Procesando 1086-i_h-egg.egg
🔊 Procesando 1086-i_l-egg.egg
🔊 Procesando 1086-i_lhl-egg.egg
🔊 Procesando 1086-i_n-egg.egg
🔊 Procesando 1086-u_h-egg.egg
🔊 Procesando 1086-u_l-egg.egg
🔊 Procesando 1086-u_lhl-egg.egg
🔊 Procesando 1086-u_n-egg.egg
🔊 Procesando 1194-a_h-egg.egg
🔊 Procesando 1194-a_l-egg.egg
🔊 Procesando 1194-a_lhl-egg.egg
🔊 Procesando 1194-a_n-egg.egg
🔊 Procesando 1194-iau-egg.egg
🔊 Procesando 1194-i_h-egg.egg
🔊 Procesando 1194-i_l-egg.egg
🔊 Procesando 1194-i_lhl-egg.egg
🔊 Procesando 1194-i_n-egg.egg
🔊 Procesando 1194-u_h-egg.egg
🔊 Procesando 1194-u_l-egg.egg
🔊 Procesando 1194-u_lhl-egg.egg
🔊 Procesando 1194-u_n-egg.egg
🔊 Procesando 1219-a_h-egg.egg
🔊 Procesando 1219-a_l-egg.egg
🔊 Procesando 1219-a_lhl-egg.egg
🔊 Procesando 1219-a_n-egg.egg
🔊 Procesando 1219-iau-egg.egg
🔊 Procesando 1219-i_h-egg.egg
🔊 Procesando 1219-i_l-egg.egg
🔊 Procesando 1219-i_lhl-egg.egg
🔊 Procesando 1219-i_n-egg.egg
🔊 Procesando 1219-u_h-egg.